In [1]:
import numpy as np
from network import Network
from fc_layer import FCLayer
from activation_layer import ActivationLayer
from activation_function import tanh, tanh_prime, soft_max
from loss_function import mse, mse_prime
from copy import deepcopy
import random
from typing import List, Tuple
from crossover import crossover
from mutation import mutate
from mate import mate
import pandas as pd
import matplotlib.pyplot as plt
from helper_module import split_train_set, split_train_test_sets


# training data
x_train = np.array([[[0,0]], [[0,1]], [[1,0]], [[1,1]]])
y_train = np.array([[[0]], [[1]], [[1]], [[0]]])

# network
# net = Network()
# net.add(FCLayer(2, 3))
# net.add(ActivationLayer(tanh, tanh_prime))
# net.add(FCLayer(3, 1))
# net.add(ActivationLayer(tanh, tanh_prime))

# # train
# net.use(mse, mse_prime)
# net.fit(x_train, y_train, epochs=1000, learning_rate=0.1)
# # test
# out = net.predict(x_train)
# print(out)

In [ ]:
# read and print sample historical stock price data
series = pd.read_csv('../../Data/PH-historical-stock-price-data-csv/GLO.csv', header = 0, index_col = 'Date')
print(series)

# plot historical stock price data
fig, ax = plt.subplots()    
ax.set_title('GLO closing prices')
ax.set_xlabel('Date')
ax.set_ylabel('Price (Pesos)')
ax.plot(series['Close'].tail(60))
ax.set_xticks(ax.get_xticks()[::13])
plt.show()

In [ ]:
# split data into train and test sets
train, test = split_train_test_sets(series)

# plot the train and test sets
fig, ax = plt.subplots()    
ax.set_title('GLO closing prices')
ax.set_xlabel('Date')
ax.set_ylabel('Price (Pesos)')
ax.plot(train['Close'], 'g')
ax.plot(test['Close'], 'r')
ax.set_xticks(ax.get_xticks()[::2000])
plt.show()

In [ ]:
# split the train set into multiple batches
train_set = split_train_set(train)

# plot a sample of the train set
fig, ax = plt.subplots()    
ax.set_title('GLO closing prices')
ax.set_xlabel('Date')
ax.set_ylabel('Price (Pesos)')
ax.plot(train_set[1]['Close'], 'g')
ax.set_xticks(ax.get_xticks()[::100])
plt.show()

In [2]:
# network
net = Network()
net.add(FCLayer(150, 150))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(150, 150))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(150, 150))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(150, 150))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(150, 150))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(150, 150))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(150, 150))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(150, 150))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(150, 150))
net.add(ActivationLayer(tanh, tanh_prime))
net.add(FCLayer(150, 3))
net.add(ActivationLayer(soft_max))

In [3]:
def flatten_NN(NN: Network):
    
    # initialize some variables
    flat_NN = np.array([])
    num_layers: int = 0
    NN_layer_dims: List[Tuple[int, int]] = list(tuple())
    
    # loop through all the layers in the network
    for layer in net.layers:
        if isinstance(layer, FCLayer):
            # NN_layer_dims.append(tuple(int(dim)) for dim in np.shape(layer.weights)))
            NN_layer_dims.append(np.shape(layer.weights))
            flat_NN = np.append(flat_NN, layer.weights.flatten())
            flat_NN = np.append(flat_NN, layer.bias.flatten())
            num_layers += 1
    
    return flat_NN, num_layers, NN_layer_dims

In [4]:
def reconstruct_NN(flat_NN, num_layers, NN_layer_dims) -> Network:
    elems_taken = 0
    NN = Network()
    for layer_id, NN_layer_dim in zip(range(num_layers), NN_layer_dims):
        
        # instatiate variables
        rows, columns = NN_layer_dim
        fc_layer = FCLayer(rows, columns)
        layer_num_elems = rows*columns
        
        # get the weigths for the FC layer
        layer_weights_elems = flat_NN[elems_taken:elems_taken+layer_num_elems]
        elems_taken += layer_num_elems
        fc_layer.weights = np.reshape(layer_weights_elems, (rows, columns))
        
        # get the biases for the FC layer
        layer_bias_elems = flat_NN[elems_taken:elems_taken+columns]
        elems_taken += columns
        fc_layer.bias = np.reshape(layer_bias_elems, (1, columns))
        
        NN.add(fc_layer)
        
        if layer_id == num_layers-1:
            NN.add(ActivationLayer(soft_max)) 
        else:
            NN.add(ActivationLayer(tanh, tanh_prime))   
        
    return NN

In [5]:
test_flat_NN, test_num_layers, test_NN_layers_dims = flatten_NN(net)
test_reconstructed_NN = reconstruct_NN(flat_NN = test_flat_NN, num_layers = test_num_layers, NN_layer_dims = test_NN_layers_dims)

In [6]:
# for orig_NN_layer, recon_NN_layer in zip(net.layers, test_reconstructed_NN.layers):
#     if isinstance(orig_NN_layer, FCLayer) and isinstance(recon_NN_layer, FCLayer):
#         print(orig_NN_layer.weights == recon_NN_layer.weights)

In [7]:
test_sample = np.zeros([1,150])
for i in range(150):
    test_sample[0][i] = np.random.uniform(-10,10)

In [8]:
print(net.predict(test_sample))
print(test_reconstructed_NN.predict(test_sample))

[array([[0.09169166, 0.0599223 , 0.84838604]])]
[array([[0.09169166, 0.0599223 , 0.84838604]])]


In [9]:
test_array = np.zeros((5,5))
test_tuple = np.shape(test_array)
x, y = test_tuple
print(type(x), type(y))

<class 'int'> <class 'int'>


In [10]:
test_offspring1, test_offspring2 = crossover(test_flat_NN, test_flat_NN)

In [11]:
reconstructed_offspring1 = reconstruct_NN(flat_NN = test_offspring1, num_layers = test_num_layers, NN_layer_dims = test_NN_layers_dims)
reconstructed_offspring2 = reconstruct_NN(flat_NN = test_offspring2, num_layers = test_num_layers, NN_layer_dims = test_NN_layers_dims)

In [12]:
print(reconstructed_offspring1)

In [13]:
mutated_offspring = mutate(test_offspring1)

In [14]:
print(mutated_offspring)

[ 0.09798846  0.13366966  0.30504472 ...  0.02362918 -0.26718814
 -0.10403652]


In [15]:
for i in range(100):
    print(i)
    test_offspring1, test_offspring2 = mate(test_flat_NN, test_flat_NN)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
